# Tutoriel 2 : Réseau de neurones à convolution
***INF889G - Vision par ordinateur (UQÀM)***

Adapté du [tutoriel 2](https://github.com/pjreddie/uwimg/blob/main/tutorial2_cnns_in_pytorch.ipynb) du cours CSE455 à l'Université de Washington (J. Redmon).

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/joe-from-mtl/teaching/blob/main/notebooks/inf889g-vision/tuto2_cnns_pytorch.ipynb)

Ce tutoriel explore les convets avec l'ensemble de données [Cifar10](https://www.cs.toronto.edu/~kriz/cifar.html). Commençons par importer les modules python utilisés par ce tutoriel et par vérifier si un GPU est accessible.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

## Obtenir les données du Cifar
C'est assez simple, on peut tout simplement utiliser les fonctionnalités intégrée de PyTorch pour construire un ensemble de données. Pour ce tutoriel, nous utiliserons également de l'augmentation de données (*data augmentation*).

**`RandomCrop(32, padding=4)`** : Cela signifie que nous allons rogner des régions 32x32 aléatoires de l'image ayant dabord été augmentée par un remplissage de zéros de taille 4 pixels (*zero padding*). Étant donné que l'image est de taille 32x32, cela signifie que nous devons d'abord faire un remplissage de zéro pour en faire une image de taille 40x40 (en ajoutant 4 pixels par côté), puis choisir aléatoirement et rogner une région 32x32. Cela signifie que le réseau voit une version de l'image légèrement décalée à chaque fois, il est donc plus difficile de surajuster des pixels spécifiques à des endroits spécifiques. Cela oblige le réseau à apprendre des filtres plus robustes et réduit le surajustement.

**`RandomHorizontalFlip()`** : Ceci signifie que la moitié du temps l'image sera retournée horizontalement. La raison est la même que plus huat, le réseau voit des versions différentes des données, ce qui rend plus difficile le surajustement.

**Note:** l'augmentation des données est déasctivé par défaut. On essaiera d'entraîner le réseau normalement, puis nous évaluerons l'effet de l'augmentation des données sur les performances.

In [ ]:
def get_cifar10_data(augmentation: bool=False) -> dict:
    # Transformations de type "augmentation des données"
    # Ce n'est pas pour les tests !
    if augmentation:
        transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4, padding_mode='edge'), # Rogner une région 32x32 depuis une image rembourrée (padded) 40x40
        transforms.RandomHorizontalFlip(),    # 50% du temps faire une réflection horizontale selon l'axe y
        transforms.ToTensor(),
        ])
    else: 
        transform_train = transforms.ToTensor()

    transform_test = transforms.Compose([
        transforms.ToTensor(),])

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True,
                                        transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True,
                                            num_workers=20)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True,
                                      transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False,
                                          num_workers=20)
    classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    return {'train': trainloader, 'test': testloader, 'classes': classes}

data = get_cifar10_data()

**Astuce pratique :** dans colab, vous pouvez exécuter des commandes dans la machine virtuelle sous-jacente (en dehors de python) en les préfixant d'un point d'exclamation, comme ceci :

`!ls ./data`

Notez que chaque fois que vous appelez une commande avec `!` Colab génère un nouveau shell. Donc les commandes comme `!cd` ne persistent pas entre les lignes.

In [ ]:
!ls ./data

On dirait que les données sont dans le bon dossier ! Pour CIFAR10, l'ensemble d'apprentissage est de 50 000 images et l'ensemble de test est de 10 000 :

In [ ]:
print(data['train'].__dict__)
print(data['test'].__dict__)

### Inspection des données 

In [ ]:
dataiter = iter(data['train'])
images, labels = dataiter.next()
print(images.size())

def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Afficher les images
imshow(torchvision.utils.make_grid(images))

# Afficher les étiquettes
print("Labels:" + ' '.join('%9s' % data['classes'][labels[j]] for j in range(8)))

flat = torch.flatten(images, 1)
print(images.size())
print(flat.size())

## Définir le réseau

### SimpleNet
Essayons d'abord le réseau simple `SimpleNet` utilisé lors du premier tutoriel. Note : le nombre d'entrées a changé puisque les images en entrée sont maintenant des images RGB de taille 32x32 (32x32x3 = 3072).

In [ ]:
class SimpleNet(nn.Module):
    def __init__(self, inputs=3072, hidden=512, outputs=10):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(inputs, hidden)
        self.fc2 = nn.Linear(hidden, outputs)

    def forward(self, x):
        x = torch.flatten(x, 1) # Convertit une image en vecteur
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

### CNN

Notre réseau de neurones à convolution est simple pour commencer. Il a 3 couches de convolution (`conv{1-3}`) suivi d'une couche entièrement connectée (`fc1`).

|Couche|Entrée|Filtres|Sortie|
|-|-|-|-|
|**conv1**|`32x32x3`|16 filtres `3x3`, stride=2|`16x16x16`|
|**conv2**|`16x16x16`|32 filtres `3x3`, stride=2|`8x8x32`|
|**conv3**|`8x8x32`|64 filtres `3x3`, stride=2|`4x4x64`|
|**fc1**|`1024`|N.A.|`10`|

La couche `fc1` recoît un vecteur de taille 1024 en entrée, et retourne en sortie un vecteur de taille 10, représentant les probabilités non-normalisée de chaque classe.

**Note :** après la 3e couche de convolution, on doit convertir la carte de caractéristiques entre formats de tenseurs. Le résultat de cette couche est une image de format (NxCxHxW), mais la couche entièrement connectée reçoit un vecteur de taille (NxM).

On peut faire cela en utilisant la méthode `x = torch.flatten(x,1)` sur la carte de caractéristiques.

Vous pouvez aussi voir dans la méthode `forward` qu'on utilise une fonction d'activation `relu` après chaque convolution.

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__() # https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
        # Entrée : image 32x32x3
        # 16 filtres de taille 3x3x3 (ils ont aussi 3 canaux)
        # stride 2 (sous-échantillonne d'un facteur 2)
        # Sortie : image 16x16x16
        self.conv1 = nn.Conv2d(3, 16, 3, stride=2, padding=1)

        # Entrée : image 16x16x16
        # 32 filtres de taille 3x3x16 filter size (ils ont aussi 16 canaux)
        # stride 2 (sous-échantillonne d'un facteur 2)
        # Sortie : image 8x8x32
        self.conv2 = nn.Conv2d(16, 32, 3, stride=2, padding=1)

        # Entrée : image 8x8x32
        # 64 filtres de taille 3x3x32 filter size (ils ont aussi 32 canaux)
        # stride 2 (sous-échantillonne d'un facteur 2)
        # Sortie : image 4x4x64
        # Output image: 4x4x64 -> 1024 neurons
        self.conv3 = nn.Conv2d(32, 64, 3, stride=2, padding=1)

        # Couche entièrement connectée
        self.fc1 = nn.Linear(1024, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = torch.flatten(x, 1) # Conversion de l'image 4x4x64 -> 1024 neurones
        x = self.fc1(x)
        return x


### Code d'entraînement

Même boucle d'entraînement que pour le premier tutoriel.

In [ ]:
def train(net, dataloader, epochs=1, lr=0.01, momentum=0.9, decay=0.0, verbose=1):
    net.to(device)
    losses = []
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum, weight_decay=decay)
    for epoch in range(epochs):
        sum_loss = 0.0
        for i, batch in enumerate(dataloader, 0):
            # Obtenir les entrées; les données forme une liste de [inputs, labels]
            inputs, labels = batch[0].to(device), batch[1].to(device)

            # Initialiser les paramètres des gradients à zéro
            optimizer.zero_grad()

            # Propagation avant (forward) + Rétropropagation (backward) + Optimisation 
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()  # autograd magic ! Calcule toutes les dérivées partielles
            optimizer.step() # Effectue un pas dans la direction du gradient

            # Affiche des statistiques
            losses.append(loss.item())
            sum_loss += loss.item()
            if i % 100 == 99:    # Affiche chaque 100 mini-batches
                if verbose:
                    print('[%d, %5d] loss: %.3f' %
                         (epoch + 1, i + 1, sum_loss / 100))
                sum_loss = 0.0
    return losses

def accuracy(net, dataloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in dataloader:
            images, labels = batch[0].to(device), batch[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct/total


def smooth(x, size):
    return np.convolve(x, np.ones(size)/size, mode='valid')

## Entraînement des réseaux !

### SimpleNet avec Cifar10

In [ ]:
net = SimpleNet(inputs=3072)

losses = train(net, data['train'], epochs=5, lr=.01)
plt.plot(smooth(losses,50))

print("Précision d'entraînement : %f" % accuracy(net, data['train']))
print("Précision de test : %f" % accuracy(net, data['test']))

### ConvNet avec CIFAR 10

In [ ]:
conv_net = ConvNet()

conv_losses = train(conv_net, data['train'], epochs=15, lr=.01)
plt.plot(smooth(conv_losses, 50))

print("Précision d'entraînement : %f" % accuracy(conv_net, data['train']))
print("Précision de test : %f" % accuracy(conv_net, data['test']))

In [ ]:
plt.plot(smooth(losses,50), 'r-')
plt.plot(smooth(conv_losses, 50), 'b-')

### Recuit simulé (*Simulated Annealing*)
https://en.wikipedia.org/wiki/Simulated_annealing

Il peut être utile de réduire lenteement le taux d'apprentissage avec le temps pour aider le réseau à converger vers un meilleur optimum local. Essayons cette technique.

In [ ]:
anneal_net = ConvNet()

anneal_losses =  train(anneal_net, data['train'], epochs=5, lr=.1)
anneal_losses += train(anneal_net, data['train'], epochs=5, lr=.01)
anneal_losses += train(anneal_net, data['train'], epochs=5, lr=.001)

plt.plot(smooth(anneal_losses, 50))

print("Précision d'entraînement : %f" % accuracy(anneal_net, data['train']))
print("Précision de test : %f" % accuracy(anneal_net, data['test']))

### Normalisation par lot (*Batch Normalization*)
L'entraînement est meilleur et plus rapide avec `batchnorm`. Ajoutons cette couche à notre réseau.

In [ ]:
class ConvBNNet(nn.Module):
    def __init__(self):
        super(ConvBNNet, self).__init__() # https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
        self.conv1 = nn.Conv2d(3, 16, 3, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, 3, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, 3, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(1024, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x

In [ ]:
norm_net = ConvBNNet()

norm_losses = train(norm_net, data['train'], epochs=15, lr=.01)

plt.plot(smooth(norm_losses, 50))

print("Précision d'entraînement : %f" % accuracy(norm_net, data['train']))
print("Précision de test : %f" % accuracy(norm_net, data['test']))

In [ ]:
plt.plot(smooth(losses,50), 'r-', label='SimpleNet')
plt.plot(smooth(conv_losses, 50), 'b-', label='ConvNet')
plt.plot(smooth(norm_losses, 50), 'g-', label='ConvNet+BatchNorm')
plt.legend()
plt.show()

In [ ]:
# Augmentation du taux d'apprentissage

lr_net = ConvBNNet()

lr_losses = train(lr_net, data['train'], epochs=15, lr=.1)

plt.plot(smooth(lr_losses, 50))

print("Précision d'entraînement : %f" % accuracy(lr_net, data['train']))
print("Précision de test : %f" % accuracy(lr_net, data['test']))

In [ ]:
plt.plot(smooth(norm_losses, 50), 'g-', label='lr=0.01')
plt.plot(smooth(lr_losses, 50), 'r-', label='lr=0.1')
plt.legend()
plt.show()

In [ ]:
# ConvNet + BatchNorm + simulated annealing

anneal2_net = ConvBNNet()

anneal2_losses =  train(anneal2_net, data['train'], epochs=5, lr=.1)
anneal2_losses += train(anneal2_net, data['train'], epochs=5, lr=.01)
anneal2_losses += train(anneal2_net, data['train'], epochs=5, lr=.001)


plt.plot(smooth(anneal2_losses, 50))

print("Précision d'entraînement : %f" % accuracy(anneal2_net, data['train']))
print("Précision de test : %f" % accuracy(anneal2_net, data['test']))

### Décroissance des poids (*Weight Decay*)

On peut essayer d'ajouter une décroissance des poids, car le modèle actuel surajuste les données.

In [ ]:
decay_net = ConvBNNet()

decay_losses =  train(decay_net, data['train'], epochs=5, lr=.1  , decay = .0005)
decay_losses += train(decay_net, data['train'], epochs=5, lr=.01 , decay = .0005)
decay_losses += train(decay_net, data['train'], epochs=5, lr=.001, decay = .0005)


plt.plot(smooth(decay_losses, 50))

print("Précision d'entraînement : %f" % accuracy(decay_net, data['train']))
print("Précision de test : %f" % accuracy(decay_net, data['test']))

In [ ]:
#plt.plot(smooth(losses,50), 'r-')
#plt.plot(smooth(conv_losses, 50), 'r-')
#plt.plot(smooth(norm_losses, 50), 'g-')
plt.plot(smooth(anneal2_losses, 50), 'b-')
plt.plot(smooth(decay_losses, 50), 'm-')

#### Augmentation des données (*Data Augmentation*)

La précision pour l'entraînement est beaucoup plus élevée que la précision pour les données de test, ce qui indique un surajustement. Ajoutoins l'augmentation des données pour l'entraînement.

In [ ]:
data_aug = get_cifar10_data(augmentation=True)
data_net = ConvBNNet()

data_losses =  train(data_net, data_aug['train'], epochs=5, lr=.1  , decay=.0005)
data_losses += train(data_net, data_aug['train'], epochs=5, lr=.01 , decay=.0005)
data_losses += train(data_net, data_aug['train'], epochs=5, lr=.001, decay=.0005)


plt.plot(smooth(data_losses, 50))

print("Précision d'entraînement : %f" % accuracy(data_net, data_aug['train']))
print("Précision de test : %f" % accuracy(data_net, data_aug['test']))

In [ ]:
plt.plot(smooth(decay_losses, 50), 'r-')
plt.plot(smooth(data_losses, 50), 'g-')

In [ ]:
final_net = ConvBNNet()

final_losses =  train(final_net, data_aug['train'], epochs=15, lr=.1  , decay=.0005)
final_losses += train(final_net, data_aug['train'], epochs=5, lr=.01 , decay=.0005)
final_losses += train(final_net, data_aug['train'], epochs=5, lr=.001, decay=.0005)


plt.plot(smooth(final_losses, 50))

print("Précision d'entraînement : %f" % accuracy(final_net, data_aug['train']))
print("Précision de test : %f" % accuracy(final_net, data_aug['test']))

In [ ]:
plt.plot(smooth(decay_losses, 50), 'r-')
plt.plot(smooth(data_losses, 50), 'g-')
plt.plot(smooth(final_losses, 50), 'b-')

# Tester le réseau

In [ ]:
import random
dataiter = iter(data['test'])
images, labels = dataiter.next()

idx = random.randint(0, len(images-4))
images = images[idx:idx+4]
labels = labels[idx:idx+4]

# Afficher les images
imshow(torchvision.utils.make_grid(images))
print('Vérité de terrain (GroundTruth): ', ' '.join('%5s' % data['classes'][labels[j]] for j in range(4)))
outputs = final_net(images.to(device))
_, predicted = torch.max(outputs, 1)

print('Prédiction : ', ' '.join('%5s' % data['classes'][predicted[j]]
                              for j in range(4)))